In [5]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime 

import sys,os
sys.path.append('py/') 

import shutil
from functions import initialize, get_points_within_target_region, df2gdf, plot_map, check_dir, get_all_files_from_dir
from preprocess import preprocess_data
from map_matching import map_match_csv2gpx, map_match_csv2gpx_multithread
from generate_route_by_pyroutelib import generate_osm_routes_main
from config import max_threads, OUTPUT_DIR, INPUT_DIR

- csv files from  **....path/heromiya/PHLMobilityData/**  contains many fields.
- Interesting fields in each csv are got via: 
         cut -d ',' -f 1,2,3,4,6,7,8,9 PDP_philippines_20110101.csv >  PHL_20110101.csv
- timestamp field is creadted by joining different fileds using **py/preprocess_raw.py** ( a copy is kept workstation ....path/bidur/PHL_raw_data/preprocess_raw.py )
    - csv files with desired fields *(ap_id, timestamp, latitude, longitude)* are saved at : **....path/bidur/PHL_raw_data/clean_input/**


In [ ]:
input_dir =  '....path/bidur/PHL_raw_data/clean_input/'
backup_dir = '....path/bidur/PHL_raw_data/csv_4_mobmap/'

In [3]:

#backup_dir = '/home/bidur/map_match_gps_data/raw_data/phl_sample/csv_4_mobmap/'
#input_dir = '/home/bidur/map_match_gps_data/raw_data/phl_sample/clean/'  

arr_input_csv = get_all_files_from_dir(input_dir)
print(arr_input_csv)
for gps_csv in arr_input_csv:
    
    print(gps_csv, 'START ', str( datetime.now() ))
 
    # 1. remove old data and create necessary directories
    initialize()

    # 2. ananymize ap_id column to int value ,   clip points within boundary
    gdf_probe_clipped, gdf_target = get_points_within_target_region (gps_csv, anonymize=True, display_plot = False)
    

    # 3. Preprocess: cleaning data & applying sampling
    df_sample = preprocess_data()
    print('----3 done----')
    display(df_sample.head())

    # 4. map matching with osm roads using graphhopper
    df_mapped_route = map_match_csv2gpx_multithread(df_sample) # multithreaded
    
    
    
    #Use this for normal execution without multi thread:
    #df_mapped_route = map_match_csv2gpx(df_sample)
    print('----4 done----')
    
        
    # 5. save to backup dir
    csv_path, csv_name = os.path.split(gps_csv)
    final_data_dir = backup_dir+csv_name.replace('.csv','')
    print(final_data_dir)
    check_dir(final_data_dir)# create subdir for this iteration
    
    shutil.copytree(INPUT_DIR, final_data_dir +'/input')
    shutil.copytree(OUTPUT_DIR, final_data_dir +'/output')
    
    print(gps_csv, 'END ', str( datetime.now() ))
    print('__________________________________________________________________')
    


"\n#backup_dir = '/home/bidur/map_match_gps_data/raw_data/phl_sample/csv_4_mobmap/'\n#input_dir = '/home/bidur/map_match_gps_data/raw_data/phl_sample/clean/'  \n\narr_input_csv = get_all_files_from_dir(input_dir)\nprint(arr_input_csv)\nfor gps_csv in arr_input_csv:\n    \n    print(gps_csv, 'START ', str( datetime.now() ))\n \n    # 1. remove old data and create necessary directories\n    initialize()\n\n    # 2. ananymize ap_id column to int value ,   clip points within boundary\n    gdf_probe_clipped, gdf_target = get_points_within_target_region (gps_csv, anonymize=True, display_plot = False)\n    \n\n    # 3. Preprocess: cleaning data & applying sampling\n    df_sample = preprocess_data()\n    print('----3 done----')\n    display(df_sample.head())\n\n    # 4. map matching with osm roads using graphhopper\n    df_mapped_route = map_match_csv2gpx_multithread(df_sample) # multithreaded\n    \n    \n    \n    #Use this for normal execution without multi thread:\n    #df_mapped_route = m

#### Output summary for a sample inout for 20190703
- about 9 hr 30 min 
- 10% = 525 ap_ids
- 411 ap_ids in final output 


In [4]:
!pwd

/home/bidur/map_match_gps_data
